In [1]:
import json


In [2]:
json_files = ["alpaca_train_data.json", "alpaca_valid_data.json", "alpaca_test_data.json"]
def modify_json(json_file):
    with open( json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    for record in data:
        # Overwrite instruction with your fixed message
        record["instruction"] = "you are a cyber security expert that can detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode."
    
        input_text = record.get("input", "").strip()
        lines = input_text.splitlines()
    
        if lines:
            last_line = lines.pop().strip()
    
        numbered_lines = [last_line+ "\n"] +  [f"{i + 1}: {line}" for i, line in enumerate(lines)]
        record["input"] = "\n".join(numbered_lines)
    return data
    

vul_data = []
for json_file in json_files:
    data = modify_json(json_file)
    vul_data.extend(data)
    
# with open(f"corrected_v1/{json_file}", "w", encoding="utf-8") as f:
#     json.dump(data, f, ensure_ascii=False, indent=2)


In [3]:
# i = 500
# print(vul_data[i]['instruction'])
# print(vul_data[i]['input'])
# print(vul_data[i]['output'])

In [4]:
td_intro = "find all tiemstamp dependency vulnerabilities and detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode based on the instruction."
re_intro = "find all Reentrancy vulnerabilities and detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode based on the instruction."
iou_intro = "find all integeroverflow/underflow vulnerabilities and detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode based on the instruction."

In [5]:
import json

jsonl_files = ["safe_TD.jsonl", "safe_RE.jsonl", "safe_IoU.jsonl"]
intros = [td_intro, re_intro, iou_intro]


def gpt_to_alpaca(gpt_dict):
    messages = gpt_dict.get("messages", [])
    instruction = ""
    input_content = ""
    output = ""
    for msg in messages:
        if msg["role"] == "system":
            instruction = msg["content"]
        elif msg["role"] == "user":
            input_content = msg["content"]
        elif msg["role"] == "assistant":
            output = msg["content"]
    return {
        "instruction": instruction.strip(),
        "input": input_content.strip(),
        "output": output.strip()
    }


safe_data = []
for jsonl_file, intro in zip(jsonl_files, intros):
    data = []
    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            parsed = json.loads(line)
            converted = gpt_to_alpaca(parsed)
            data.append(converted)


    for record in data:
        record['input'] = intro + "\n\n" +  "\n".join([f"{i + 1}: {line}" for i, line in enumerate(record["input"].split("\n"))]) 
    safe_data.extend(data)
safe_data = [rec for rec in safe_data if len(rec['input'].split("\n")) <= 600]


In [6]:

for rec in safe_data:
    input = rec['input']
    count = len(input.split("\n"))
    if count>600:
        print(count)

In [7]:
# i = 1000
# print(safe_data[i]['instruction'])
# print(safe_data[i]['input'])
# print(safe_data[i]['output'])

In [8]:
dataset = vul_data + safe_data

In [12]:
import random

random.shuffle(dataset)

train_split = 0.8
val_split = 0.10
test_split = 0.10

n = len(dataset)
train_end = int(n * train_split)
val_end = train_end + int(n * val_split)

train_data = dataset[:train_end]
val_data = dataset[train_end:val_end]
test_data = dataset[val_end:]


In [18]:
with open(f"corrected_v1/alpaca_train_smart.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)


with open(f"corrected_v1/alpaca_val_smart.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)


with open(f"corrected_v1/alpaca_test_smart.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

In [21]:
import json

with open('vulnerability_instruction.txt', 'r') as txt_file, open('vulnerability_instruction.jsonl', 'w') as jsonl_file:
    for line in txt_file:
        data = {
            "instruction": "Provide information on the following vulnerability.",
            "input": line.strip(),
            "output": ""
        }
        jsonl_file.write(json.dumps(data) + '\n')


In [27]:
with open("corrected_v1/alpaca_test_smart.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("corrected_v1/alpaca_test_smart_sample.json", "w", encoding="utf-8") as f:
    json.dump(data[:10], f, ensure_ascii=False, indent=2)

In [24]:
len(data)

191

In [118]:
import json
pred_data = []
with open("corrected_v1/generated_predictions.jsonl", "r", encoding="utf-8") as f:
    for line in f:
       pred_data.append(json.loads(line))
a = pred_data[4]

In [119]:
print(a['prompt'])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

you are a cyber security expert that can detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode.
find all Reentrancy vulnerabilities and detect vulnerable line numbers from the contract as well as vulnerableCode, vulnerabilityReason, potentialSecurityRisk, and fixedCode based on the instruction.

1: pragma solidity ^0.4.19;
2: 
3: contract PENNY_BY_PENNY  
4: {
5:     struct Holder   
6:     {
7:         uint unlockTime;
8:         uint balance;
9:     }
10:     
11:     mapping (address => Holder) public Acc;
12:     
13:     uint public MinSum;
14:     
15:     LogFile Log;
16:     
17:     bool intitalized;
18:     
19:     function SetMinSum(uint _val)
20:     public
21:     {
22:         if(intitalized)throw;
23:         MinSum = _val;
24:     }
25:     
26:     function SetLogFile(address _log)
27:     public
28:     {
29:         if(intitalize

In [120]:
print(a['label'])

{
    "vulnerabilities": [
        {
            "vulnerableLines": "57-59",
            "vulnerableCode": [
                "            if(msg.sender.call.value(_am)())",
                "            {",
                "                acc.balance-=_am;"
            ],
            "vulnerabilityReason": "The function attempts to send Ether to'msg.sender' using 'call.value(_am)()' before deducting the amount from 'acc.balance'. This opens the door for a reentrancy attack where an attacker could invoke the 'Collect' function recursively and drain the contract's funds before the balance is updated.",
            "potentialSecurityRisk": "An attacker could exploit this vulnerability by calling the 'Collect' function through a malicious contract with a fallback function, allowing them to withdraw more Ether than intended before the balance is reduced, potentially leading to significant financial loss for the contract owner.",
            "fixedCode": "            acc.balance -= _am;  // 

In [121]:
print(a['predict'])

{
    "vulnerabilities": [
        {
            "vulnerableLines": "57-57",
            "vulnerableCode": [
                "            if(msg.sender.call.value(_am)())"
            ],
            "vulnerabilityReason": "The contract performs an external call to `msg.sender` using `call.value(_am)()` before updating the user's balance. This allows a malicious contract to re-enter the `Collect` function before the balance is decremented, potentially allowing it to withdraw more funds than it should.",
            "potentialSecurityRisk": "An attacker could exploit this vulnerability by deploying a malicious contract that re-enters the `Collect` function during the external call, allowing it to drain funds from the contract before the balance is updated.",
            "fixedCode": "            acc.balance -= _am; // Update balance before external call\n            if(!msg.sender.call.value(_am)()) { // External call after balance update\n                acc.balance += _am; // Revert ba